# Getting started: Classification

This tutorial uses safeds on **titanic passenger data** to predict who will survive and who will not, using sex as a feature for the prediction.


1. Load your data into a `Table`, the data is available under `docs/tutorials/data/titanic.csv`:


In [20]:
from safeds.data.tabular.containers import Table

titanic = Table.from_csv_file("data/titanic.csv")
#For visualisation purposes we only print out the first 15 rows.
titanic.slice_rows(0,15)

,id,name,sex,age,siblings_spouses,parents_children,ticket,travel_class,fare,cabin,port_embarked,survived
0,1,"Abbott, Master. Eugene Joseph",male,13.0000,0,2,C.A. 2673,3,20.2500,NaN,Southampton,0
1,2,"Abbott, Mr. Rossmore Edward",male,16.0000,1,1,C.A. 2673,3,20.2500,NaN,Southampton,0
2,3,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0000,1,1,C.A. 2673,3,20.2500,NaN,Southampton,1
3,4,"Abelseth, Miss. Karen Marie",female,16.0000,0,0,348125,3,7.6500,NaN,Southampton,1
4,5,"Abelseth, Mr. Olaus Jorgensen",male,25.0000,0,0,348122,3,7.6500,F G63,Southampton,1
5,6,"Abelson, Mr. Samuel",male,30.0000,1,0,P/PP 3381,2,24.0000,NaN,Cherbourg,0
6,7,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0000,1,0,P/PP 3381,2,24.0000,NaN,Cherbourg,1
7,8,"Abrahamsson, Mr. Abraham August Johannes",male,20.0000,0,0,SOTON/O2 3101284,3,7.9250,NaN,Southampton,1
8,9,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0000,0,0,2657,3,7.2292,NaN,Cherbourg,1
9,10,"Adahl, Mr. Mauritz Nils Martin",male,30.0000,0,0,C 7076,3,7.2500,NaN,Southampton,0


2. Split the titanic dataset into two tables. A training set, that we will use later to implement a training model to predict the survival of passengers, containing 60% of the data, and a testing set containing the rest of the data.
Delete the column `survived` from the test set, to be able to predict it later:

In [21]:
split_tuple = titanic.split(0.60)

train_table = split_tuple[0]
testing_table = split_tuple[1]

test_table = testing_table.remove_columns(["survived"]).shuffle_rows()

3. Use `OneHotEncoder` to create an encoder, that will be used later to transform the training table.
* We use `OneHotEncoder` to transform non-numerical categorical values into numerical representations with values of zero or one. In this example we will transform the values of the sex column, hence they will be used in the model for predicting the surviving of passengers.
* Use the `fit` function of the `OneHotEncoder` to pass the table and the column names, that will be used as features to predict who will survive to the encoder.
* The names of the column before transformation need to be saved, because `OneHotEncoder` changes the names of the fitted `Column`s:


In [22]:
from safeds.data.tabular.transformation import OneHotEncoder

old_column_names = train_table.column_names
encoder = OneHotEncoder().fit(train_table, ["sex"])

4. Transform the training table using the fitted encoder, and create a set with the new names of the fitted `Column`s:


In [23]:
transformed_table = encoder.transform(train_table)
new_column_names = transformed_table.column_names
new_columns= set(new_column_names) - set(old_column_names)

5. Tag the `survived` `Column` as the target variable to be predicted. Use the new names of the fitted `Column`s as features, which will be used to make predictions based on the target variable.

In [24]:
tagged_train_table= transformed_table.tag_columns("survived", feature_names=[
    *new_columns
])

6. Use `RandomForest` classifier as a model for the classification. Pass the "tagged_titanic" table to the fit function of the model:

In [25]:
from safeds.ml.classical.classification import RandomForest

model = RandomForest()
fitted_model= model.fit(tagged_train_table)

7. Use the fitted random forest model, that we trained on the training dataset to predict the survival rate of passengers in the test dataset.
Transform the test data with `OneHotEncoder` first, to be able to pass it to the predict function, that uses our fitted random forest model for prediction:

In [26]:
encoder = OneHotEncoder().fit(test_table, ["sex"])
transformed_test_table = encoder.transform(test_table)

predicition = fitted_model.predict(
    transformed_test_table
)
#For visualisation purposes we only print out the first 15 rows.
predicition.slice_rows(0,15)

,id,name,sex__male,sex__female,age,siblings_spouses,parents_children,ticket,travel_class,fare,cabin,port_embarked,survived
0,786,"McGovern, Miss. Mary",0.0,1.0,NaN,0,0,330931,3,7.8792,NaN,Queenstown,1
1,787,"McGowan, Miss. Anna 'Annie'",0.0,1.0,15.0,0,0,330923,3,8.0292,NaN,Queenstown,1
2,788,"McGowan, Miss. Katherine",0.0,1.0,35.0,0,0,9232,3,7.7500,NaN,Queenstown,1
3,789,"McKane, Mr. Peter David",1.0,0.0,46.0,0,0,28403,2,26.0000,NaN,Southampton,0
4,790,"McMahon, Mr. Martin",1.0,0.0,NaN,0,0,370372,3,7.7500,NaN,Queenstown,0
5,791,"McNamee, Mr. Neal",1.0,0.0,24.0,1,0,376566,3,16.1000,NaN,Southampton,0
6,792,"McNamee, Mrs. Neal (Eileen O'Leary)",0.0,1.0,19.0,1,0,376566,3,16.1000,NaN,Southampton,1
7,793,"McNeill, Miss. Bridget",0.0,1.0,NaN,0,0,370368,3,7.7500,NaN,Queenstown,1
8,794,"Meanwell, Miss. (Marion Ogden)",0.0,1.0,NaN,0,0,SOTON/O.Q. 392087,3,8.0500,NaN,Southampton,1
9,795,"Meek, Mrs. Thomas (Annie Louise Rowley)",0.0,1.0,NaN,0,0,343095,3,8.0500,NaN,Southampton,1


7. You can test the accuracy of that model with the initial testing_table as follows:

In [27]:
encoder = OneHotEncoder().fit(test_table, ["sex"])
testing_table = encoder.transform(testing_table)

tagged_test_table= testing_table.tag_columns("survived", feature_names=[
    *new_columns
])
fitted_model.accuracy(tagged_train_table)


0.7745222929936306